In [20]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import argparse
from mpmath import li
import sympy
from functools import lru_cache
import numpy as np
from scipy import stats



@lru_cache(maxsize=None)
def is_prime(num):
    return sympy.isprime(num)


def addition(number_to_generate_primes):
    """Generate primes based on the given number."""
    quotient = number_to_generate_primes
    secondmult = 0
    thirdmult = 0
    primes = []
    multiple_minus_one = 0
    twins_number = 0
    possible_twin1=0
    possible_twin2=0
    for index in range(2, number_to_generate_primes + 1):
        if((index & 1) == 0 or (quotient & 1) == 0 or ((index & 0) == 0 and (quotient & 0) == 0)):
            prime_plus_one = index * quotient + 1
            prime_minus_one = index * quotient - 1
            prime1=False
            prime2=False
            if (prime_plus_one&1==1 and is_prime(prime_plus_one)) or prime_plus_one==2:
                secondmult += 1
                prime1=True
                possible_twin1=prime_plus_one
                primes.append(prime_plus_one)

            if (prime_minus_one&1==1 and is_prime(prime_minus_one)) or prime_minus_one==2:
                multiple_minus_one += 1
                prime2=True
                possible_twin2=prime_minus_one
                primes.append(prime_minus_one)
            if(prime1==False or prime2==False):
                continue
            if((prime_plus_one-1) % 6!=0):
                continue
            if  (prime_plus_one-1) %6 ==0 and prime1 and prime2 and (possible_twin1 - possible_twin2 == 2):#all twin except 3,5 the composed number between the is multiple of 6n
                twins_number += 1# the 6n verification used for faster exit if not

    return secondmult, primes, multiple_minus_one, twins_number

def get_background_color(label):
    """Get the background color for a given label."""
    background_color_chart = [
        "rgba(0, 0, 0, 1.0)",  # black
        "rgba(0, 255, 0, 1.0)",  # Green (Cool)
        "rgba(255, 0, 0, 1.0)",  # Red (Warm)
        "rgba(0, 0, 255, 1.0)",  # Blue (Cool)
        "rgba(255, 0, 255, 1.0)",  # Magenta (Cool)
        "rgba(255, 255, 0, 1.0)",  # Yellow (Warm)
    ]
    background_color_chart_special = [
        "rgba(255, 128, 0, 1.0)",
        "rgba(64, 224, 208, 1.0)",
    ]

    remainder_420 = label % 210
    if remainder_420 == 0:
        return background_color_chart_special[1]
    remainder_60 = label % 30
    if remainder_60 == 0:
        return background_color_chart_special[0]
    remainder = label % 6

    if remainder == 1:
        return background_color_chart[0]  # black for 12n+2
    elif remainder == 2:
        return background_color_chart[1]  # Green for 12n+4
    elif remainder == 3:
        return background_color_chart[2]  # Red for 12n+6
    elif remainder == 4:
        return background_color_chart[3]  # Blue for 12n+8
    elif remainder == 5:
        return background_color_chart[4]  # Magenta for 12n+10
    elif remainder == 0:
        return background_color_chart[5]  # Yellow for 12n

    # Default color for any unexpected cases
    return "rgba(128, 128, 128, 1.0)"  # Gray

def calculate_range(start, end):
    """Calculate prime-related data for a range of numbers."""
    results = []
    for number in range(start, end + 1):
        result = addition(number)
        prime_check = is_prime(number)
        results.append((number, result, prime_check))
    return results

def calculate(min_interval, end_interval):
    """Perform the main calculations and generate the data for plotting."""
    unique_primes = []
    labels = []
    labels2 = []
    labels3 = []
    labels4 = []
    prime_count = []
    primes_count_plus = 0
    primes_count_minus = 0
    prime_count_minus_one_equation = []
    prime_twins_count = []
    prime_twins_exception = []
    prime_twins_exception3 = []
    prime_twins_exception4 = []
    prime_twins_ratio_plus_one = []
    prime_reverse_twins_ratio_plus_one = []
    ln_func = []
    ln_func2 = []
    ln_func3 = []
    ln_func4 = []
    ln = []
    labels_six_plus_one=[]
    values_six_plus_one=[]
    reciproc_ln_2=[]
    reciproc_ln_4=[]
    reciproc_ln_for_multiple_of_3 = []
    reciproc_ln = []
    prime_ratio=[]
    sqrt2 = math.sqrt(2)
    # Parallel processing
    chunk_size = 200  # Adjust the chunk size based on your system's capabilities
    ranges = [(i, min(i + chunk_size - 1, end_interval)) for i in range(min_interval, end_interval + 1, chunk_size)]
    print(ranges)

    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(calculate_range, start, end) for start, end in ranges]
        for future in as_completed(futures):
            results = future.result()
            for number, result, prime_check in results:
                if number > 0:  # Ensure number is positive
                    labels.append(number)
                    prime_count.append(result[0])
                    primes_count_plus += result[0]
                    primes_count_minus += result[2]
                    prime_count_minus_one_equation.append(result[2])
                    prime_twins_count.append(result[3])
                    #print(number,result[0],result[4])
                    math_log=math.log(number)
                    divide_log=round(number / math_log)
                    ln_func.append(divide_log)
                    ln_func2.append(divide_log * sqrt2)
                    ln_func3.append(divide_log / sqrt2)
                    ln_func4.append(divide_log * 2 * sqrt2)
                    log_number = math_log - np.pi
                    log_number_pi_2 = math_log - 2*math.sqrt(2)
                    log_number_pi_4 = math_log + 2*math.sqrt(2)
                    result_log_twin_prime_ratio = log_number
                    
                    ln.append(result_log_twin_prime_ratio)
                    reciproc_ln_for_multiple_of_3.append(result_log_twin_prime_ratio - 2)
                    reciproc_ln.append(1 / result_log_twin_prime_ratio)
                    reciproc_ln_2.append(1 / log_number_pi_2)
                    reciproc_ln_4.append(1 / log_number_pi_4)
                    prime_ratio.append((result[3]/number)*100)
                    if(result[3]>0):
                        prime_twins_ratio_plus_one.append(result[0] / result[3])
                    else:
                        prime_twins_ratio_plus_one.append(0)
                    prime_reverse_twins_ratio_plus_one.append(result[3] / result[0])

                    if number % 6 == 0 and result[3] > 0:
                        labels2.append(number)
                        prime_twins_exception.append(result[3])
                    if number % 30 == 0 and result[3] > 0:
                        labels3.append(number)
                        prime_twins_exception3.append(result[3])
                    if number % 210 == 0 and result[3] > 0:
                        labels4.append(number)
                        prime_twins_exception4.append(result[3])
                    if number % 6 == 1 and result[3] > 0:
                        labels_six_plus_one.append(number)
                        values_six_plus_one.append(result[3])
                    

                    


    # Plotting
    fig = go.Figure()

    # Add scatter plot for prime count
    fig.add_trace(go.Scatter(
        x=labels,
        y=prime_count,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# of primes equation +1'
    ))

    # Add line plots for ln functions
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func,
        mode='lines',
        line=dict(color='black'),
        name='# of primes equation +1'
    ))
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func2,
        mode='lines',
        line=dict(color='green'),
        name='# of primes equation +1 *sqrt'
    ))
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func3,
        mode='lines',
        line=dict(color='yellow'),
        name='# of primes equation +1 /sqrt'
    ))
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func4,
        mode='lines',
        line=dict(color='blue'),
        name='# of primes equation +1 *2sqrt'
    ))

    fig.update_layout(title='Prime Count and Ln Functions', xaxis_title='Number', yaxis_title='Prime Count')
    fig.show()

    # Plot for prime count minus one equation
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=labels,
        y=prime_count_minus_one_equation,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# of primes equation -1'
    ))
    fig2.add_trace(go.Scatter(
        x=labels,
        y=ln_func,
        mode='lines',
        line=dict(color='black'),
        name='# of primes equation +1'
    ))
    fig2.update_layout(title='Prime Count Minus One Equation', xaxis_title='Number', yaxis_title='Prime Count')
    fig2.show()

    # Bar plot for prime twins exception
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(
        x=labels2,
        y=prime_twins_exception,
        marker=dict(color=[get_background_color(label) for label in labels2]),
        name='# twins'
    ))
    fig3.update_layout(title='Prime Twins Exception', xaxis_title='Number', yaxis_title='Twin Primes Count')
    fig3.show()

    fig_six_plus_on = go.Figure()
    fig_six_plus_on.add_trace(go.Scatter(
        x=labels_six_plus_one,
        y=values_six_plus_one,
        mode="markers",
        marker=dict(color="black"),
        name='# twins'
    ))
    fig_six_plus_on.update_layout(title='values of 6+1', xaxis_title='Number', yaxis_title='Primes Count')
    fig_six_plus_on.show()

    # Scatter plot for prime twins exception 3
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(
        x=labels3,
        y=prime_twins_exception3,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels3]),
        name='# twins'
    ))
    fig4.update_layout(title='Prime Twins Exception 3', xaxis_title='Number', yaxis_title='Twin Primes Count')
    fig4.show()

    # Bar plot for prime twins exception 4
    fig5 = go.Figure()
    fig5.add_trace(go.Bar(
        x=labels4,
        y=prime_twins_exception4,
        marker=dict(color=[get_background_color(label) for label in labels4]),
        name='# twins'
    ))
    fig5.update_layout(title='Prime Twins Exception 4', xaxis_title='Number', yaxis_title='Twin Primes Count')
    fig5.show()

    # Scatter plot for prime twins ratio plus one
    fig6 = go.Figure()
    fig6.add_trace(go.Scatter(
        x=labels,
        y=prime_twins_ratio_plus_one,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# ratio'
    ))
    fig6.add_trace(go.Scatter(
        x=labels,
        y=ln,
        mode='lines',
        line=dict(color='black'),
        name='# of primes equation +1'
    ))
    fig6.update_layout(title='Prime Twins Ratio Plus One', xaxis_title='Number', yaxis_title='Ratio')
    fig6.show()

    # Scatter plot for prime reverse twins ratio plus one
    fig7 = go.Figure()
    fig7.add_trace(go.Scatter(
        x=labels,
        y= prime_reverse_twins_ratio_plus_one,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# ratio'
    ))
    fig7.add_trace(go.Scatter(
        x=labels,
        y= reciproc_ln,
        mode='lines',
        line=dict(color='red'),
        name='1/(ln(x)-pi)'
    ))
    fig7.add_trace(go.Scatter(
        x=labels,
        y= reciproc_ln_2,
        mode='lines',
        line=dict(color='blue'),
        name='1/(ln(x)-pi/2)'
    ))
    fig7.add_trace(go.Scatter(
        x=labels,
        y= reciproc_ln_4,
        mode='lines',
        line=dict(color='red'),
        name='1/(ln(x)+pi)'
    ))
    fig7.update_layout(title='Prime Reverse Twins Ratio Plus One', xaxis_title='Number', yaxis_title='Ratio')
    fig7.show()
    fig8 = go.Figure()
    fig8.add_trace(go.Scatter(
        x=labels,
        y= prime_ratio,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='twin primes per number percentage'
    ))
    fig8.update_layout(title='twin primes per number percentage', xaxis_title='Number', yaxis_title='percentage')
    fig8.show()
    # Observed values (prime twin ratios)

    """observed_ratios_plus_one_6n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 6 == 0])
    expected_reciproc_ln_6n = np.array([ratio for number, ratio in zip(labels, reciproc_ln) if number % 6 == 0])

    # Calculate residuals for reciproc_ln (multiples of 6n)
    residuals_reciproc_ln_6n = observed_ratios_plus_one_6n - expected_reciproc_ln_6n

    # RMSE calculations for reciproc_ln (multiples of 6n)
    rmse_reciproc_ln_6n = np.sqrt(np.mean(residuals_reciproc_ln_6n**2))

    print("RMSE for reciproc_ln (multiples of 6n):", rmse_reciproc_ln_6n)

    # R-squared calculations for reciproc_ln (multiples of 6n)
    ss_res_reciproc_ln_6n = np.sum(residuals_reciproc_ln_6n**2)
    ss_tot_reciproc_ln_6n = np.sum((observed_ratios_plus_one_6n - np.mean(observed_ratios_plus_one_6n))**2)
    r_squared_reciproc_ln_6n = 1 - (ss_res_reciproc_ln_6n / ss_tot_reciproc_ln_6n)

    print("R-squared for reciproc_ln (multiples of 6n):", r_squared_reciproc_ln_6n)

    # Kolmogorov-Smirnov test for reciproc_ln (multiples of 6n)
    ks_stat_reciproc_ln_6n, ks_p_value_reciproc_ln_6n = stats.ks_2samp(observed_ratios_plus_one_6n, expected_reciproc_ln_6n)

    print("K-S test for reciproc_ln (multiples of 6n):", ks_stat_reciproc_ln_6n, ks_p_value_reciproc_ln_6n)

    # Fitness test for reciproc_ln_4 (not multiples of 6n)
    observed_ratios_plus_one_not_6n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 6 != 0])
    expected_reciproc_ln_4_not_6n = np.array([ratio for number, ratio in zip(labels, reciproc_ln_4) if number % 6 != 0])

    # Calculate residuals for reciproc_ln_4 (not multiples of 6n)
    residuals_reciproc_ln_4_not_6n = observed_ratios_plus_one_not_6n - expected_reciproc_ln_4_not_6n

    # RMSE calculations for reciproc_ln_4 (not multiples of 6n)
    rmse_reciproc_ln_4_not_6n = np.sqrt(np.mean(residuals_reciproc_ln_4_not_6n**2))

    print("RMSE for reciproc_ln_4 (not multiples of 6n):", rmse_reciproc_ln_4_not_6n)

    # R-squared calculations for reciproc_ln_4 (not multiples of 6n)
    ss_res_reciproc_ln_4_not_6n = np.sum(residuals_reciproc_ln_4_not_6n**2)
    ss_tot_reciproc_ln_4_not_6n = np.sum((observed_ratios_plus_one_not_6n - np.mean(observed_ratios_plus_one_not_6n))**2)
    r_squared_reciproc_ln_4_not_6n = 1 - (ss_res_reciproc_ln_4_not_6n / ss_tot_reciproc_ln_4_not_6n)

    print("R-squared for reciproc_ln_4 (not multiples of 6n):", r_squared_reciproc_ln_4_not_6n)

    # Kolmogorov-Smirnov test for reciproc_ln_4 (not multiples of 6n)
    ks_stat_reciproc_ln_4_not_6n, ks_p_value_reciproc_ln_4_not_6n = stats.ks_2samp(observed_ratios_plus_one_not_6n, expected_reciproc_ln_4_not_6n)

    print("K-S test for reciproc_ln_4 (not multiples of 6n):", ks_stat_reciproc_ln_4_not_6n, ks_p_value_reciproc_ln_4_not_6n)






    print("************************************Second")

    # Fitness test for reciproc_ln (multiples of 3n)
    observed_ratios_plus_one_3n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 6 == 0 and number % 30 !=0])
    expected_reciproc_ln_3n = np.array([ratio for number, ratio in zip(labels, reciproc_ln) if number % 6 == 0 and number % 30 !=0])

    # Calculate residuals for reciproc_ln (multiples of 3n)
    residuals_reciproc_ln_3n = observed_ratios_plus_one_3n - expected_reciproc_ln_3n

    # RMSE calculations for reciproc_ln (multiples of 3n)
    rmse_reciproc_ln_3n = np.sqrt(np.mean(residuals_reciproc_ln_3n**2))
    print("RMSE for reciproc_ln (multiples of 3n):", rmse_reciproc_ln_3n)

    # R-squared calculations for reciproc_ln (multiples of 3n)
    ss_res_reciproc_ln_3n = np.sum(residuals_reciproc_ln_3n**2)
    ss_tot_reciproc_ln_3n = np.sum((observed_ratios_plus_one_3n - np.mean(observed_ratios_plus_one_3n))**2)
    r_squared_reciproc_ln_3n = 1 - (ss_res_reciproc_ln_3n / ss_tot_reciproc_ln_3n)
    print("R-squared for reciproc_ln (multiples of 3n):", r_squared_reciproc_ln_3n)

    # Kolmogorov-Smirnov test for reciproc_ln (multiples of 3n)
    ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n = stats.ks_2samp(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("K-S test for reciproc_ln (multiples of 3n):", ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n)

    # Fitness test for reciproc_ln_4 (not multiples of 3n)
    observed_ratios_plus_one_not_3n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 3 != 0])
    expected_reciproc_ln_4_not_3n = np.array([ratio for number, ratio in zip(labels, reciproc_ln_4) if number % 3 != 0])

    # Calculate residuals for reciproc_ln_4 (not multiples of 3n)
    residuals_reciproc_ln_4_not_3n = observed_ratios_plus_one_not_3n - expected_reciproc_ln_4_not_3n

    # RMSE calculations for reciproc_ln_4 (not multiples of 3n)
    rmse_reciproc_ln_4_not_3n = np.sqrt(np.mean(residuals_reciproc_ln_4_not_3n**2))
    print("RMSE for reciproc_ln_4 (not multiples of 3n):", rmse_reciproc_ln_4_not_3n)

    # R-squared calculations for reciproc_ln_4 (not multiples of 3n)
    ss_res_reciproc_ln_4_not_3n = np.sum(residuals_reciproc_ln_4_not_3n**2)
    ss_tot_reciproc_ln_4_not_3n = np.sum((observed_ratios_plus_one_not_3n - np.mean(observed_ratios_plus_one_not_3n))**2)
    r_squared_reciproc_ln_4_not_3n = 1 - (ss_res_reciproc_ln_4_not_3n / ss_tot_reciproc_ln_4_not_3n)
    print("R-squared for reciproc_ln_4 (not multiples of 3n):", r_squared_reciproc_ln_4_not_3n)

    # Kolmogorov-Smirnov test for reciproc_ln_4 (not multiples of 3n)
    ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n = stats.ks_2samp(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("K-S test for reciproc_ln_4 (not multiples of 3n):", ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n)

    # Additional Analysis: Absolute Means
    mean_observed_3n = np.mean(observed_ratios_plus_one_3n)
    mean_expected_3n = np.mean(expected_reciproc_ln_3n)
    mean_observed_not_3n = np.mean(observed_ratios_plus_one_not_3n)
    mean_expected_not_3n = np.mean(expected_reciproc_ln_4_not_3n)

    print("Mean of observed_ratios_plus_one_3n:", mean_observed_3n)
    print("Mean of expected_reciproc_ln_3n:", mean_expected_3n)
    print("Mean of observed_ratios_plus_one_not_3n:", mean_observed_not_3n)
    print("Mean of expected_reciproc_ln_4_not_3n:", mean_expected_not_3n)

    # Perform t-tests to compare means
    t_stat_3n, p_value_3n = stats.ttest_ind(observed_ratios_plus_one_3n, expected_reciproc_ln_3n, equal_var=False)
    t_stat_not_3n, p_value_not_3n = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, equal_var=False)

    print("T-test for means (multiples of 3n): t-statistic =", t_stat_3n, ", p-value =", p_value_3n)
    print("T-test for means (not multiples of 3n): t-statistic =", t_stat_not_3n, ", p-value =", p_value_not_3n)
    mean_observed_3n = np.mean(observed_ratios_plus_one_3n)
    mean_expected_3n = np.mean(expected_reciproc_ln_3n)
    median_observed_3n = np.median(observed_ratios_plus_one_3n)
    median_expected_3n = np.median(expected_reciproc_ln_3n)
    var_observed_3n = np.var(observed_ratios_plus_one_3n)
    var_expected_3n = np.var(expected_reciproc_ln_3n)

    # Summary statistics for not multiples of 3n
    mean_observed_not_3n = np.mean(observed_ratios_plus_one_not_3n)
    mean_expected_not_3n = np.mean(expected_reciproc_ln_4_not_3n)
    median_observed_not_3n = np.median(observed_ratios_plus_one_not_3n)
    median_expected_not_3n = np.median(expected_reciproc_ln_4_not_3n)
    var_observed_not_3n = np.var(observed_ratios_plus_one_not_3n)
    var_expected_not_3n = np.var(expected_reciproc_ln_4_not_3n)

    print(f"Multiples of 3n - Observed Mean: {mean_observed_3n}, Expected Mean: {mean_expected_3n}")
    print(f"Multiples of 3n - Observed Median: {median_observed_3n}, Expected Median: {median_expected_3n}")
    print(f"Multiples of 3n - Observed Variance: {var_observed_3n}, Expected Variance: {var_expected_3n}")

    print(f"Not Multiples of 3n - Observed Mean: {mean_observed_not_3n}, Expected Mean: {mean_expected_not_3n}")
    print(f"Not Multiples of 3n - Observed Median: {median_observed_not_3n}, Expected Median: {median_expected_not_3n}")
    print(f"Not Multiples of 3n - Observed Variance: {var_observed_not_3n}, Expected Variance: {var_expected_not_3n}")
    # T-test for multiples of 3n
    t_stat_3n, p_value_3n = stats.ttest_ind(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print(f"T-test for multiples of 3n: t-statistic = {t_stat_3n}, p-value = {p_value_3n}")
    # Mann-Whitney U Test for multiples of 3n
    u_stat_3n, p_value_u_3n = stats.mannwhitneyu(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print(f"Mann-Whitney U Test for multiples of 3n: U-statistic = {u_stat_3n}, p-value = {p_value_u_3n}")

    # Mann-Whitney U Test for not multiples of 3n
    u_stat_not_3n, p_value_u_not_3n = stats.mannwhitneyu(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print(f"Mann-Whitney U Test for not multiples of 3n: U-statistic = {u_stat_not_3n}, p-value = {p_value_u_not_3n}")

    # T-test for not multiples of 3n
    t_stat_not_3n, p_value_not_3n = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print(f"T-test for not multiples of 3n: t-statistic = {t_stat_not_3n}, p-value = {p_value_not_3n}")

    levene_test = stats.levene(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Levene’s Test: Statistic = {}, p-value = {}".format(levene_test.statistic, levene_test.pvalue))
    bartlett_test = stats.bartlett(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Bartlett’s Test: Statistic = {}, p-value = {}".format(bartlett_test.statistic, bartlett_test.pvalue))
    brown_forsythe_test = stats.levene(observed_ratios_plus_one_3n, expected_reciproc_ln_3n, center='median')
    print("Brown-Forsythe Test: Statistic = {}, p-value = {}".format(brown_forsythe_test.statistic, brown_forsythe_test.pvalue))
    fligner_killeen_test = stats.fligner(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Fligner-Killeen Test: Statistic = {}, p-value = {}".format(fligner_killeen_test.statistic, fligner_killeen_test.pvalue))
    welch_t_test = stats.ttest_ind(observed_ratios_plus_one_3n, expected_reciproc_ln_3n, equal_var=False)
    print("Welch’s T-Test: t-statistic = {}, p-value = {}".format(welch_t_test.statistic, welch_t_test.pvalue))
    def z_test_variance(obs, exp):
        var_obs = np.var(obs, ddof=1)  # Variance of observed data
        var_exp = np.var(exp, ddof=1)  # Variance of expected data
        z_stat = (var_obs - var_exp) / np.sqrt((2 * (var_obs**2 / len(obs)) + 2 * (var_exp**2 / len(exp))))
        return z_stat

    z_statistic = z_test_variance(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Z-Test for Variance Ratios: Z-statistic = {}".format(z_statistic))
    def cochran_c_test(obs, exp):
        var_obs = np.var(obs, ddof=1)
        var_exp = np.var(exp, ddof=1)
        max_var = max(var_obs, var_exp)
        sum_var = var_obs + var_exp
        c_statistic = max_var / sum_var
        return c_statistic

    cochran_c_statistic = cochran_c_test(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Cochran’s C Test: C-statistic = {}".format(cochran_c_statistic))
    kruskal_wallis_test = stats.kruskal(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Kruskal-Wallis Test: Statistic = {}, p-value = {}".format(kruskal_wallis_test.statistic, kruskal_wallis_test.pvalue))
    levene_test = stats.levene(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Levene’s Test: Statistic = {}, p-value = {}".format(levene_test.statistic, levene_test.pvalue))
    bartlett_test = stats.bartlett(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Bartlett’s Test: Statistic = {}, p-value = {}".format(bartlett_test.statistic, bartlett_test.pvalue))
    brown_forsythe_test = stats.levene(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, center='median')
    print("Brown-Forsythe Test: Statistic = {}, p-value = {}".format(brown_forsythe_test.statistic, brown_forsythe_test.pvalue))
    fligner_killeen_test = stats.fligner(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Fligner-Killeen Test: Statistic = {}, p-value = {}".format(fligner_killeen_test.statistic, fligner_killeen_test.pvalue))
    welch_t_test = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, equal_var=False)
    print("Welch’s T-Test: t-statistic = {}, p-value = {}".format(welch_t_test.statistic, welch_t_test.pvalue))
    def z_test_variance(obs, exp):
        var_obs = np.var(obs, ddof=1)  # Variance of observed data
        var_exp = np.var(exp, ddof=1)  # Variance of expected data
        z_stat = (var_obs - var_exp) / np.sqrt((2 * (var_obs**2 / len(obs)) + 2 * (var_exp**2 / len(exp))))
        return z_stat

    z_statistic = z_test_variance(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Z-Test for Variance Ratios: Z-statistic = {}".format(z_statistic))
    def cochran_c_test(obs, exp):
        var_obs = np.var(obs, ddof=1)
        var_exp = np.var(exp, ddof=1)
        max_var = max(var_obs, var_exp)
        sum_var = var_obs + var_exp
        c_statistic = max_var / sum_var
        return c_statistic

    cochran_c_statistic = cochran_c_test(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Cochran’s C Test: C-statistic = {}".format(cochran_c_statistic))
    kruskal_wallis_test = stats.kruskal(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Kruskal-Wallis Test: Statistic = {}, p-value = {}".format(kruskal_wallis_test.statistic, kruskal_wallis_test.pvalue))


    print("********************************reciproc")
    def compute_reciprocals(arr):
        arr = np.array(arr)  # Ensure input is a NumPy array
        return np.where(arr != 0, 1 / arr, np.inf)  # Replace zeros with infinity to avoid division errors
    reciprocals_observed_ratios_3n = compute_reciprocals(observed_ratios_plus_one_3n)
    reciprocals_expected_reciproc_ln_3n = compute_reciprocals(expected_reciproc_ln_3n)


    # Calculate residuals for reciproc_ln (multiples of 3n)
    residuals_reciproc_ln_3n = reciprocals_observed_ratios_3n - reciprocals_expected_reciproc_ln_3n

    # RMSE calculations for reciproc_ln (multiples of 3n)
    rmse_reciproc_ln_3n = np.sqrt(np.mean(residuals_reciproc_ln_3n**2))
    print("RMSE for reciproc_ln (multiples of 3n):", rmse_reciproc_ln_3n)

    # R-squared calculations for reciproc_ln (multiples of 3n)
    ss_res_reciproc_ln_3n = np.sum(residuals_reciproc_ln_3n**2)
    ss_tot_reciproc_ln_3n = np.sum((reciprocals_observed_ratios_3n - np.mean(reciprocals_observed_ratios_3n))**2)
    r_squared_reciproc_ln_3n = 1 - (ss_res_reciproc_ln_3n / ss_tot_reciproc_ln_3n)
    print("R-squared for reciproc_ln (multiples of 3n):", r_squared_reciproc_ln_3n)

    # Kolmogorov-Smirnov test for reciproc_ln (multiples of 3n)
    ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n = stats.ks_2samp(reciprocals_observed_ratios_3n, reciprocals_expected_reciproc_ln_3n)
    print("K-S test for reciproc_ln (multiples of 3n):", ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n)

    # Fitness test for reciproc_ln_4 (not multiples of 3n)
    observed_ratios_plus_one_not_3n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 3 != 0])
    expected_reciproc_ln_4_not_3n = np.array([ratio for number, ratio in zip(labels, reciproc_ln_4) if number % 3 != 0])

    # Calculate residuals for reciproc_ln_4 (not multiples of 3n)
    residuals_reciproc_ln_4_not_3n = observed_ratios_plus_one_not_3n - expected_reciproc_ln_4_not_3n

    # RMSE calculations for reciproc_ln_4 (not multiples of 3n)
    rmse_reciproc_ln_4_not_3n = np.sqrt(np.mean(residuals_reciproc_ln_4_not_3n**2))
    print("RMSE for reciproc_ln_4 (not multiples of 3n):", rmse_reciproc_ln_4_not_3n)

    # R-squared calculations for reciproc_ln_4 (not multiples of 3n)
    ss_res_reciproc_ln_4_not_3n = np.sum(residuals_reciproc_ln_4_not_3n**2)
    ss_tot_reciproc_ln_4_not_3n = np.sum((observed_ratios_plus_one_not_3n - np.mean(observed_ratios_plus_one_not_3n))**2)
    r_squared_reciproc_ln_4_not_3n = 1 - (ss_res_reciproc_ln_4_not_3n / ss_tot_reciproc_ln_4_not_3n)
    print("R-squared for reciproc_ln_4 (not multiples of 3n):", r_squared_reciproc_ln_4_not_3n)

    # Kolmogorov-Smirnov test for reciproc_ln_4 (not multiples of 3n)
    ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n = stats.ks_2samp(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("K-S test for reciproc_ln_4 (not multiples of 3n):", ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n)

    # Additional Analysis: Absolute Means
    mean_observed_3n = np.mean(reciprocals_observed_ratios_3n)
    mean_expected_3n = np.mean(reciprocals_expected_reciproc_ln_3n)
    mean_observed_not_3n = np.mean(observed_ratios_plus_one_not_3n)
    mean_expected_not_3n = np.mean(expected_reciproc_ln_4_not_3n)

    print("Mean of reciprocals_observed_ratios_3n:", mean_observed_3n)
    print("Mean of reciprocals_expected_reciproc_ln_3n:", mean_expected_3n)
    print("Mean of observed_ratios_plus_one_not_3n:", mean_observed_not_3n)
    print("Mean of expected_reciproc_ln_4_not_3n:", mean_expected_not_3n)

    # Perform t-tests to compare means
    t_stat_3n, p_value_3n = stats.ttest_ind(reciprocals_observed_ratios_3n, reciprocals_expected_reciproc_ln_3n, equal_var=False)
    t_stat_not_3n, p_value_not_3n = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, equal_var=False)

    print("T-test for means (multiples of 3n): t-statistic =", t_stat_3n, ", p-value =", p_value_3n)
    print("T-test for means (not multiples of 3n): t-statistic =", t_stat_not_3n, ", p-value =", p_value_not_3n)
    """

def main():
    min_interval = int(input("Enter the minimum interval: "))
    end_interval = int(input("Enter the maximum interval: "))
    start_time = time.time()
    calculate(min_interval, end_interval)
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time} seconds")

if __name__ == "__main__":
    main()


[(60000, 60100)]


Elapsed time: 64.76813697814941 seconds


In [1]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import argparse
from mpmath import li
import sympy
from functools import lru_cache
import numpy as np
from scipy import stats



@lru_cache(maxsize=None)
def is_prime(num):
    return sympy.isprime(num)


def addition(number_to_generate_primes):
    """Generate primes based on the given number."""
    quotient = number_to_generate_primes
    secondmult = 0
    thirdmult = 0
    primes = []
    multiple_minus_one = 0
    twins_number = 0
    possible_twin1=0
    possible_twin2=0
    for index in range(2, number_to_generate_primes + 1):
        if((index & 1) == 0 or (quotient & 1) == 0):
            prime_plus_one = index * quotient + 1
            prime_minus_one = index * quotient - 1
            prime1=False
            prime2=False
            if (prime_plus_one&1==1 and is_prime(prime_plus_one)) or prime_plus_one==2:
                secondmult += 1
                prime1=True
                possible_twin1=prime_plus_one
                primes.append(prime_plus_one)

            if (prime_minus_one&1==1 and is_prime(prime_minus_one)) or prime_minus_one==2:
                multiple_minus_one += 1
                prime2=True
                possible_twin2=prime_minus_one
                primes.append(prime_minus_one)
            if(prime1==False or prime2==False):
                continue
            if((prime_plus_one-1) % 6!=0):
                continue
            if  (prime_plus_one-1) %6 ==0 and prime1 and prime2 and (possible_twin1 - possible_twin2 == 2):#all twin except 3,5 the composed number between the is multiple of 6n
                twins_number += 1# the 6n verification used for faster exit if not

    return secondmult, primes, multiple_minus_one, twins_number

def get_background_color(label):
    """Get the background color for a given label."""
    background_color_chart = [
        "rgba(0, 0, 0, 1.0)",  # black
        "rgba(0, 255, 0, 1.0)",  # Green (Cool)
        "rgba(255, 0, 0, 1.0)",  # Red (Warm)
        "rgba(0, 0, 255, 1.0)",  # Blue (Cool)
        "rgba(255, 0, 255, 1.0)",  # Magenta (Cool)
        "rgba(255, 255, 0, 1.0)",  # Yellow (Warm)
    ]
    background_color_chart_special = [
        "rgba(255, 128, 0, 1.0)",
        "rgba(64, 224, 208, 1.0)",
    ]

    remainder_420 = label % 210
    if remainder_420 == 0:
        return background_color_chart_special[1]
    remainder_60 = label % 30
    if remainder_60 == 0:
        return background_color_chart_special[0]
    remainder = label % 6

    if remainder == 1:
        return background_color_chart[0]  # black for 12n+2
    elif remainder == 2:
        return background_color_chart[1]  # Green for 12n+4
    elif remainder == 3:
        return background_color_chart[2]  # Red for 12n+6
    elif remainder == 4:
        return background_color_chart[3]  # Blue for 12n+8
    elif remainder == 5:
        return background_color_chart[4]  # Magenta for 12n+10
    elif remainder == 0:
        return background_color_chart[5]  # Yellow for 12n

    # Default color for any unexpected cases
    return "rgba(128, 128, 128, 1.0)"  # Gray

def calculate_range(start, end):
    """Calculate prime-related data for a range of numbers."""
    results = []
    for number in range(start, end + 1):
        result = addition(number)
        prime_check = is_prime(number)
        results.append((number, result, prime_check))
    return results

def calculate(min_interval, end_interval):
    """Perform the main calculations and generate the data for plotting."""
    unique_primes = []
    labels = []
    labels2 = []
    labels3 = []
    labels4 = []
    prime_count = []
    primes_count_plus = 0
    primes_count_minus = 0
    prime_count_minus_one_equation = []
    prime_twins_count = []
    prime_twins_exception = []
    prime_twins_exception3 = []
    prime_twins_exception4 = []
    prime_twins_ratio_plus_one = []
    prime_reverse_twins_ratio_plus_one = []
    ln_func = []
    ln_func2 = []
    ln_func3 = []
    ln_func4 = []
    ln = []
    labels_six_plus_one=[]
    values_six_plus_one=[]
    reciproc_ln_2=[]
    reciproc_ln_4=[]
    reciproc_ln_for_multiple_of_3 = []
    reciproc_ln = []
    is_number_prime=[]
    sqrt2 = math.sqrt(2)
    # Parallel processing
    chunk_size = 700  # Adjust the chunk size based on your system's capabilities
    ranges = [(i, min(i + chunk_size - 1, end_interval)) for i in range(min_interval, end_interval + 1, chunk_size)]
    print(ranges)


    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(calculate_range, start, end) for start, end in ranges]
        for future in as_completed(futures):
            results = future.result()
            for number, result, prime_check in results:
                if number > 0:  # Ensure number is positive
                    labels.append(number)
                    prime_count.append(result[0])
                    primes_count_plus += result[0]
                    primes_count_minus += result[2]
                    prime_count_minus_one_equation.append(result[2])
                    prime_twins_count.append(result[3])
                    is_number_prime.append(prime_check)
                    #print(number,result[0],result[4])
                    math_log=math.log(number)
                    divide_log=round(number / math_log)
                    ln_func.append(divide_log)
                    ln_func2.append(divide_log * sqrt2)
                    ln_func3.append(divide_log / sqrt2)
                    ln_func4.append(divide_log * 2 * sqrt2)
                    log_number = math_log - np.pi
                    log_number_pi_2 = math_log - 2*math.sqrt(2)
                    log_number_pi_4 = math_log + 2*math.sqrt(2)
                    result_log_twin_prime_ratio = log_number
                    
                    ln.append(result_log_twin_prime_ratio)
                    reciproc_ln_for_multiple_of_3.append(result_log_twin_prime_ratio - 2)
                    reciproc_ln.append(1 / result_log_twin_prime_ratio)
                    reciproc_ln_2.append(1 / log_number_pi_2)
                    reciproc_ln_4.append(1 / log_number_pi_4)
                    if(result[3]>0):
                        prime_twins_ratio_plus_one.append(result[0] / result[3])
                    else:
                        prime_twins_ratio_plus_one.append(0)
                    prime_reverse_twins_ratio_plus_one.append(result[3] / result[0])

                    if number % 6 == 0 and result[3] > 0:
                        labels2.append(number)
                        prime_twins_exception.append(result[3])
                    if number % 30 == 0 and result[3] > 0:
                        labels3.append(number)
                        prime_twins_exception3.append(result[3])
                    if number % 210 == 0 and result[3] > 0:
                        labels4.append(number)
                        prime_twins_exception4.append(result[3])
                    if number % 6 == 1 and result[3] > 0:
                        labels_six_plus_one.append(number)
                        values_six_plus_one.append(result[3])
                    

                    

    data_to_csv={
        'Number':labels,
        'Primes +1':prime_count,
        'Primes -1':prime_count_minus_one_equation,
        'Twins':prime_twins_count,
        'isPrime':is_number_prime,
    }
    df_new=pd.DataFrame(data_to_csv)
    df_new.to_csv(f'primes_data{min_interval}-{end_interval}.csv',index=False)
    
    # Plotting
    """
    fig = go.Figure()

    # Add scatter plot for prime count
    fig.add_trace(go.Scatter(
        x=labels,
        y=prime_count,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# of primes equation +1'
    ))

    # Add line plots for ln functions
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func,
        mode='lines',
        line=dict(color='black'),
        name='# of primes equation +1'
    ))
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func2,
        mode='lines',
        line=dict(color='green'),
        name='# of primes equation +1 *sqrt'
    ))
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func3,
        mode='lines',
        line=dict(color='yellow'),
        name='# of primes equation +1 /sqrt'
    ))
    fig.add_trace(go.Scatter(
        x=labels,
        y=ln_func4,
        mode='lines',
        line=dict(color='blue'),
        name='# of primes equation +1 *2sqrt'
    ))
    
    fig.update_layout(title='Prime Count and Ln Functions', xaxis_title='Number', yaxis_title='Prime Count')
    fig.show()

    # Plot for prime count minus one equation
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=labels,
        y=prime_count_minus_one_equation,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# of primes equation -1'
    ))
    fig2.add_trace(go.Scatter(
        x=labels,
        y=ln_func,
        mode='lines',
        line=dict(color='black'),
        name='# of primes equation +1'
    ))
    fig2.update_layout(title='Prime Count Minus One Equation', xaxis_title='Number', yaxis_title='Prime Count')
    fig2.show()

    # Bar plot for prime twins exception
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(
        x=labels2,
        y=prime_twins_exception,
        marker=dict(color=[get_background_color(label) for label in labels2]),
        name='# twins'
    ))
    fig3.update_layout(title='Prime Twins Exception', xaxis_title='Number', yaxis_title='Twin Primes Count')
    fig3.show()

    fig_six_plus_on = go.Figure()
    fig_six_plus_on.add_trace(go.Scatter(
        x=labels_six_plus_one,
        y=values_six_plus_one,
        mode="markers",
        marker=dict(color="black"),
        name='# twins'
    ))
    fig_six_plus_on.update_layout(title='values of 6+1', xaxis_title='Number', yaxis_title='Primes Count')
    fig_six_plus_on.show()

    # Scatter plot for prime twins exception 3
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(
        x=labels3,
        y=prime_twins_exception3,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels3]),
        name='# twins'
    ))
    fig4.update_layout(title='Prime Twins Exception 3', xaxis_title='Number', yaxis_title='Twin Primes Count')
    fig4.show()

    # Bar plot for prime twins exception 4
    fig5 = go.Figure()
    fig5.add_trace(go.Bar(
        x=labels4,
        y=prime_twins_exception4,
        marker=dict(color=[get_background_color(label) for label in labels4]),
        name='# twins'
    ))
    fig5.update_layout(title='Prime Twins Exception 4', xaxis_title='Number', yaxis_title='Twin Primes Count')
    fig5.show()

    # Scatter plot for prime twins ratio plus one
    fig6 = go.Figure()
    fig6.add_trace(go.Scatter(
        x=labels,
        y=prime_twins_ratio_plus_one,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# ratio'
    ))
    fig6.add_trace(go.Scatter(
        x=labels,
        y=ln,
        mode='lines',
        line=dict(color='black'),
        name='# of primes equation +1'
    ))
    fig6.update_layout(title='Prime Twins Ratio Plus One', xaxis_title='Number', yaxis_title='Ratio')
    fig6.show()

    # Scatter plot for prime reverse twins ratio plus one
    fig7 = go.Figure()
    fig7.add_trace(go.Scatter(
        x=labels,
        y= prime_reverse_twins_ratio_plus_one,
        mode='markers',
        marker=dict(color=[get_background_color(label) for label in labels]),
        name='# ratio'
    ))
    fig7.add_trace(go.Scatter(
        x=labels,
        y= reciproc_ln,
        mode='lines',
        line=dict(color='red'),
        name='1/(ln(x)-pi)'
    ))
    fig7.add_trace(go.Scatter(
        x=labels,
        y= reciproc_ln_2,
        mode='lines',
        line=dict(color='blue'),
        name='1/(ln(x)-pi/2)'
    ))
    fig7.add_trace(go.Scatter(
        x=labels,
        y= reciproc_ln_4,
        mode='lines',
        line=dict(color='red'),
        name='1/(ln(x)+pi)'
    ))
    fig7.update_layout(title='Prime Reverse Twins Ratio Plus One', xaxis_title='Number', yaxis_title='Ratio')
    fig7.show()
# Observed values (prime twin ratios)

    observed_ratios_plus_one_6n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 6 == 0])
    expected_reciproc_ln_6n = np.array([ratio for number, ratio in zip(labels, reciproc_ln) if number % 6 == 0])

    # Calculate residuals for reciproc_ln (multiples of 6n)
    residuals_reciproc_ln_6n = observed_ratios_plus_one_6n - expected_reciproc_ln_6n

    # RMSE calculations for reciproc_ln (multiples of 6n)
    rmse_reciproc_ln_6n = np.sqrt(np.mean(residuals_reciproc_ln_6n**2))

    print("RMSE for reciproc_ln (multiples of 6n):", rmse_reciproc_ln_6n)

    # R-squared calculations for reciproc_ln (multiples of 6n)
    ss_res_reciproc_ln_6n = np.sum(residuals_reciproc_ln_6n**2)
    ss_tot_reciproc_ln_6n = np.sum((observed_ratios_plus_one_6n - np.mean(observed_ratios_plus_one_6n))**2)
    r_squared_reciproc_ln_6n = 1 - (ss_res_reciproc_ln_6n / ss_tot_reciproc_ln_6n)

    print("R-squared for reciproc_ln (multiples of 6n):", r_squared_reciproc_ln_6n)

    # Kolmogorov-Smirnov test for reciproc_ln (multiples of 6n)
    ks_stat_reciproc_ln_6n, ks_p_value_reciproc_ln_6n = stats.ks_2samp(observed_ratios_plus_one_6n, expected_reciproc_ln_6n)

    print("K-S test for reciproc_ln (multiples of 6n):", ks_stat_reciproc_ln_6n, ks_p_value_reciproc_ln_6n)

    # Fitness test for reciproc_ln_4 (not multiples of 6n)
    observed_ratios_plus_one_not_6n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 6 != 0])
    expected_reciproc_ln_4_not_6n = np.array([ratio for number, ratio in zip(labels, reciproc_ln_4) if number % 6 != 0])

    # Calculate residuals for reciproc_ln_4 (not multiples of 6n)
    residuals_reciproc_ln_4_not_6n = observed_ratios_plus_one_not_6n - expected_reciproc_ln_4_not_6n

    # RMSE calculations for reciproc_ln_4 (not multiples of 6n)
    rmse_reciproc_ln_4_not_6n = np.sqrt(np.mean(residuals_reciproc_ln_4_not_6n**2))

    print("RMSE for reciproc_ln_4 (not multiples of 6n):", rmse_reciproc_ln_4_not_6n)

    # R-squared calculations for reciproc_ln_4 (not multiples of 6n)
    ss_res_reciproc_ln_4_not_6n = np.sum(residuals_reciproc_ln_4_not_6n**2)
    ss_tot_reciproc_ln_4_not_6n = np.sum((observed_ratios_plus_one_not_6n - np.mean(observed_ratios_plus_one_not_6n))**2)
    r_squared_reciproc_ln_4_not_6n = 1 - (ss_res_reciproc_ln_4_not_6n / ss_tot_reciproc_ln_4_not_6n)

    print("R-squared for reciproc_ln_4 (not multiples of 6n):", r_squared_reciproc_ln_4_not_6n)

    # Kolmogorov-Smirnov test for reciproc_ln_4 (not multiples of 6n)
    ks_stat_reciproc_ln_4_not_6n, ks_p_value_reciproc_ln_4_not_6n = stats.ks_2samp(observed_ratios_plus_one_not_6n, expected_reciproc_ln_4_not_6n)

    print("K-S test for reciproc_ln_4 (not multiples of 6n):", ks_stat_reciproc_ln_4_not_6n, ks_p_value_reciproc_ln_4_not_6n)






    print("************************************Second")

    # Fitness test for reciproc_ln (multiples of 3n)
    observed_ratios_plus_one_3n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 6 == 0 and number % 30 !=0])
    expected_reciproc_ln_3n = np.array([ratio for number, ratio in zip(labels, reciproc_ln) if number % 6 == 0 and number % 30 !=0])

    # Calculate residuals for reciproc_ln (multiples of 3n)
    residuals_reciproc_ln_3n = observed_ratios_plus_one_3n - expected_reciproc_ln_3n

    # RMSE calculations for reciproc_ln (multiples of 3n)
    rmse_reciproc_ln_3n = np.sqrt(np.mean(residuals_reciproc_ln_3n**2))
    print("RMSE for reciproc_ln (multiples of 3n):", rmse_reciproc_ln_3n)

    # R-squared calculations for reciproc_ln (multiples of 3n)
    ss_res_reciproc_ln_3n = np.sum(residuals_reciproc_ln_3n**2)
    ss_tot_reciproc_ln_3n = np.sum((observed_ratios_plus_one_3n - np.mean(observed_ratios_plus_one_3n))**2)
    r_squared_reciproc_ln_3n = 1 - (ss_res_reciproc_ln_3n / ss_tot_reciproc_ln_3n)
    print("R-squared for reciproc_ln (multiples of 3n):", r_squared_reciproc_ln_3n)

    # Kolmogorov-Smirnov test for reciproc_ln (multiples of 3n)
    ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n = stats.ks_2samp(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("K-S test for reciproc_ln (multiples of 3n):", ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n)

    # Fitness test for reciproc_ln_4 (not multiples of 3n)
    observed_ratios_plus_one_not_3n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 3 != 0])
    expected_reciproc_ln_4_not_3n = np.array([ratio for number, ratio in zip(labels, reciproc_ln_4) if number % 3 != 0])

    # Calculate residuals for reciproc_ln_4 (not multiples of 3n)
    residuals_reciproc_ln_4_not_3n = observed_ratios_plus_one_not_3n - expected_reciproc_ln_4_not_3n

    # RMSE calculations for reciproc_ln_4 (not multiples of 3n)
    rmse_reciproc_ln_4_not_3n = np.sqrt(np.mean(residuals_reciproc_ln_4_not_3n**2))
    print("RMSE for reciproc_ln_4 (not multiples of 3n):", rmse_reciproc_ln_4_not_3n)

    # R-squared calculations for reciproc_ln_4 (not multiples of 3n)
    ss_res_reciproc_ln_4_not_3n = np.sum(residuals_reciproc_ln_4_not_3n**2)
    ss_tot_reciproc_ln_4_not_3n = np.sum((observed_ratios_plus_one_not_3n - np.mean(observed_ratios_plus_one_not_3n))**2)
    r_squared_reciproc_ln_4_not_3n = 1 - (ss_res_reciproc_ln_4_not_3n / ss_tot_reciproc_ln_4_not_3n)
    print("R-squared for reciproc_ln_4 (not multiples of 3n):", r_squared_reciproc_ln_4_not_3n)

    # Kolmogorov-Smirnov test for reciproc_ln_4 (not multiples of 3n)
    ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n = stats.ks_2samp(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("K-S test for reciproc_ln_4 (not multiples of 3n):", ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n)

    # Additional Analysis: Absolute Means
    mean_observed_3n = np.mean(observed_ratios_plus_one_3n)
    mean_expected_3n = np.mean(expected_reciproc_ln_3n)
    mean_observed_not_3n = np.mean(observed_ratios_plus_one_not_3n)
    mean_expected_not_3n = np.mean(expected_reciproc_ln_4_not_3n)

    print("Mean of observed_ratios_plus_one_3n:", mean_observed_3n)
    print("Mean of expected_reciproc_ln_3n:", mean_expected_3n)
    print("Mean of observed_ratios_plus_one_not_3n:", mean_observed_not_3n)
    print("Mean of expected_reciproc_ln_4_not_3n:", mean_expected_not_3n)

    # Perform t-tests to compare means
    t_stat_3n, p_value_3n = stats.ttest_ind(observed_ratios_plus_one_3n, expected_reciproc_ln_3n, equal_var=False)
    t_stat_not_3n, p_value_not_3n = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, equal_var=False)

    print("T-test for means (multiples of 3n): t-statistic =", t_stat_3n, ", p-value =", p_value_3n)
    print("T-test for means (not multiples of 3n): t-statistic =", t_stat_not_3n, ", p-value =", p_value_not_3n)
    mean_observed_3n = np.mean(observed_ratios_plus_one_3n)
    mean_expected_3n = np.mean(expected_reciproc_ln_3n)
    median_observed_3n = np.median(observed_ratios_plus_one_3n)
    median_expected_3n = np.median(expected_reciproc_ln_3n)
    var_observed_3n = np.var(observed_ratios_plus_one_3n)
    var_expected_3n = np.var(expected_reciproc_ln_3n)

    # Summary statistics for not multiples of 3n
    mean_observed_not_3n = np.mean(observed_ratios_plus_one_not_3n)
    mean_expected_not_3n = np.mean(expected_reciproc_ln_4_not_3n)
    median_observed_not_3n = np.median(observed_ratios_plus_one_not_3n)
    median_expected_not_3n = np.median(expected_reciproc_ln_4_not_3n)
    var_observed_not_3n = np.var(observed_ratios_plus_one_not_3n)
    var_expected_not_3n = np.var(expected_reciproc_ln_4_not_3n)

    print(f"Multiples of 3n - Observed Mean: {mean_observed_3n}, Expected Mean: {mean_expected_3n}")
    print(f"Multiples of 3n - Observed Median: {median_observed_3n}, Expected Median: {median_expected_3n}")
    print(f"Multiples of 3n - Observed Variance: {var_observed_3n}, Expected Variance: {var_expected_3n}")

    print(f"Not Multiples of 3n - Observed Mean: {mean_observed_not_3n}, Expected Mean: {mean_expected_not_3n}")
    print(f"Not Multiples of 3n - Observed Median: {median_observed_not_3n}, Expected Median: {median_expected_not_3n}")
    print(f"Not Multiples of 3n - Observed Variance: {var_observed_not_3n}, Expected Variance: {var_expected_not_3n}")
    # T-test for multiples of 3n
    t_stat_3n, p_value_3n = stats.ttest_ind(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print(f"T-test for multiples of 3n: t-statistic = {t_stat_3n}, p-value = {p_value_3n}")
    # Mann-Whitney U Test for multiples of 3n
    u_stat_3n, p_value_u_3n = stats.mannwhitneyu(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print(f"Mann-Whitney U Test for multiples of 3n: U-statistic = {u_stat_3n}, p-value = {p_value_u_3n}")

    # Mann-Whitney U Test for not multiples of 3n
    u_stat_not_3n, p_value_u_not_3n = stats.mannwhitneyu(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print(f"Mann-Whitney U Test for not multiples of 3n: U-statistic = {u_stat_not_3n}, p-value = {p_value_u_not_3n}")

    # T-test for not multiples of 3n
    t_stat_not_3n, p_value_not_3n = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print(f"T-test for not multiples of 3n: t-statistic = {t_stat_not_3n}, p-value = {p_value_not_3n}")

    levene_test = stats.levene(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Levene’s Test: Statistic = {}, p-value = {}".format(levene_test.statistic, levene_test.pvalue))
    bartlett_test = stats.bartlett(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Bartlett’s Test: Statistic = {}, p-value = {}".format(bartlett_test.statistic, bartlett_test.pvalue))
    brown_forsythe_test = stats.levene(observed_ratios_plus_one_3n, expected_reciproc_ln_3n, center='median')
    print("Brown-Forsythe Test: Statistic = {}, p-value = {}".format(brown_forsythe_test.statistic, brown_forsythe_test.pvalue))
    fligner_killeen_test = stats.fligner(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Fligner-Killeen Test: Statistic = {}, p-value = {}".format(fligner_killeen_test.statistic, fligner_killeen_test.pvalue))
    welch_t_test = stats.ttest_ind(observed_ratios_plus_one_3n, expected_reciproc_ln_3n, equal_var=False)
    print("Welch’s T-Test: t-statistic = {}, p-value = {}".format(welch_t_test.statistic, welch_t_test.pvalue))
    def z_test_variance(obs, exp):
        var_obs = np.var(obs, ddof=1)  # Variance of observed data
        var_exp = np.var(exp, ddof=1)  # Variance of expected data
        z_stat = (var_obs - var_exp) / np.sqrt((2 * (var_obs**2 / len(obs)) + 2 * (var_exp**2 / len(exp))))
        return z_stat

    z_statistic = z_test_variance(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Z-Test for Variance Ratios: Z-statistic = {}".format(z_statistic))
    def cochran_c_test(obs, exp):
        var_obs = np.var(obs, ddof=1)
        var_exp = np.var(exp, ddof=1)
        max_var = max(var_obs, var_exp)
        sum_var = var_obs + var_exp
        c_statistic = max_var / sum_var
        return c_statistic

    cochran_c_statistic = cochran_c_test(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Cochran’s C Test: C-statistic = {}".format(cochran_c_statistic))
    kruskal_wallis_test = stats.kruskal(observed_ratios_plus_one_3n, expected_reciproc_ln_3n)
    print("Kruskal-Wallis Test: Statistic = {}, p-value = {}".format(kruskal_wallis_test.statistic, kruskal_wallis_test.pvalue))
    levene_test = stats.levene(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Levene’s Test: Statistic = {}, p-value = {}".format(levene_test.statistic, levene_test.pvalue))
    bartlett_test = stats.bartlett(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Bartlett’s Test: Statistic = {}, p-value = {}".format(bartlett_test.statistic, bartlett_test.pvalue))
    brown_forsythe_test = stats.levene(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, center='median')
    print("Brown-Forsythe Test: Statistic = {}, p-value = {}".format(brown_forsythe_test.statistic, brown_forsythe_test.pvalue))
    fligner_killeen_test = stats.fligner(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Fligner-Killeen Test: Statistic = {}, p-value = {}".format(fligner_killeen_test.statistic, fligner_killeen_test.pvalue))
    welch_t_test = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, equal_var=False)
    print("Welch’s T-Test: t-statistic = {}, p-value = {}".format(welch_t_test.statistic, welch_t_test.pvalue))
    def z_test_variance(obs, exp):
        var_obs = np.var(obs, ddof=1)  # Variance of observed data
        var_exp = np.var(exp, ddof=1)  # Variance of expected data
        z_stat = (var_obs - var_exp) / np.sqrt((2 * (var_obs**2 / len(obs)) + 2 * (var_exp**2 / len(exp))))
        return z_stat

    z_statistic = z_test_variance(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Z-Test for Variance Ratios: Z-statistic = {}".format(z_statistic))
    def cochran_c_test(obs, exp):
        var_obs = np.var(obs, ddof=1)
        var_exp = np.var(exp, ddof=1)
        max_var = max(var_obs, var_exp)
        sum_var = var_obs + var_exp
        c_statistic = max_var / sum_var
        return c_statistic

    cochran_c_statistic = cochran_c_test(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Cochran’s C Test: C-statistic = {}".format(cochran_c_statistic))
    kruskal_wallis_test = stats.kruskal(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("Kruskal-Wallis Test: Statistic = {}, p-value = {}".format(kruskal_wallis_test.statistic, kruskal_wallis_test.pvalue))


    print("********************************reciproc")
    def compute_reciprocals(arr):
        arr = np.array(arr)  # Ensure input is a NumPy array
        return np.where(arr != 0, 1 / arr, np.inf)  # Replace zeros with infinity to avoid division errors
    reciprocals_observed_ratios_3n = compute_reciprocals(observed_ratios_plus_one_3n)
    reciprocals_expected_reciproc_ln_3n = compute_reciprocals(expected_reciproc_ln_3n)


    # Calculate residuals for reciproc_ln (multiples of 3n)
    residuals_reciproc_ln_3n = reciprocals_observed_ratios_3n - reciprocals_expected_reciproc_ln_3n

    # RMSE calculations for reciproc_ln (multiples of 3n)
    rmse_reciproc_ln_3n = np.sqrt(np.mean(residuals_reciproc_ln_3n**2))
    print("RMSE for reciproc_ln (multiples of 3n):", rmse_reciproc_ln_3n)

    # R-squared calculations for reciproc_ln (multiples of 3n)
    ss_res_reciproc_ln_3n = np.sum(residuals_reciproc_ln_3n**2)
    ss_tot_reciproc_ln_3n = np.sum((reciprocals_observed_ratios_3n - np.mean(reciprocals_observed_ratios_3n))**2)
    r_squared_reciproc_ln_3n = 1 - (ss_res_reciproc_ln_3n / ss_tot_reciproc_ln_3n)
    print("R-squared for reciproc_ln (multiples of 3n):", r_squared_reciproc_ln_3n)

    # Kolmogorov-Smirnov test for reciproc_ln (multiples of 3n)
    ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n = stats.ks_2samp(reciprocals_observed_ratios_3n, reciprocals_expected_reciproc_ln_3n)
    print("K-S test for reciproc_ln (multiples of 3n):", ks_stat_reciproc_ln_3n, ks_p_value_reciproc_ln_3n)

    # Fitness test for reciproc_ln_4 (not multiples of 3n)
    observed_ratios_plus_one_not_3n = np.array([ratio for number, ratio in zip(labels, prime_reverse_twins_ratio_plus_one) if number % 3 != 0])
    expected_reciproc_ln_4_not_3n = np.array([ratio for number, ratio in zip(labels, reciproc_ln_4) if number % 3 != 0])

    # Calculate residuals for reciproc_ln_4 (not multiples of 3n)
    residuals_reciproc_ln_4_not_3n = observed_ratios_plus_one_not_3n - expected_reciproc_ln_4_not_3n

    # RMSE calculations for reciproc_ln_4 (not multiples of 3n)
    rmse_reciproc_ln_4_not_3n = np.sqrt(np.mean(residuals_reciproc_ln_4_not_3n**2))
    print("RMSE for reciproc_ln_4 (not multiples of 3n):", rmse_reciproc_ln_4_not_3n)

    # R-squared calculations for reciproc_ln_4 (not multiples of 3n)
    ss_res_reciproc_ln_4_not_3n = np.sum(residuals_reciproc_ln_4_not_3n**2)
    ss_tot_reciproc_ln_4_not_3n = np.sum((observed_ratios_plus_one_not_3n - np.mean(observed_ratios_plus_one_not_3n))**2)
    r_squared_reciproc_ln_4_not_3n = 1 - (ss_res_reciproc_ln_4_not_3n / ss_tot_reciproc_ln_4_not_3n)
    print("R-squared for reciproc_ln_4 (not multiples of 3n):", r_squared_reciproc_ln_4_not_3n)

    # Kolmogorov-Smirnov test for reciproc_ln_4 (not multiples of 3n)
    ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n = stats.ks_2samp(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n)
    print("K-S test for reciproc_ln_4 (not multiples of 3n):", ks_stat_reciproc_ln_4_not_3n, ks_p_value_reciproc_ln_4_not_3n)

    # Additional Analysis: Absolute Means
    mean_observed_3n = np.mean(reciprocals_observed_ratios_3n)
    mean_expected_3n = np.mean(reciprocals_expected_reciproc_ln_3n)
    mean_observed_not_3n = np.mean(observed_ratios_plus_one_not_3n)
    mean_expected_not_3n = np.mean(expected_reciproc_ln_4_not_3n)

    print("Mean of reciprocals_observed_ratios_3n:", mean_observed_3n)
    print("Mean of reciprocals_expected_reciproc_ln_3n:", mean_expected_3n)
    print("Mean of observed_ratios_plus_one_not_3n:", mean_observed_not_3n)
    print("Mean of expected_reciproc_ln_4_not_3n:", mean_expected_not_3n)

    # Perform t-tests to compare means
    t_stat_3n, p_value_3n = stats.ttest_ind(reciprocals_observed_ratios_3n, reciprocals_expected_reciproc_ln_3n, equal_var=False)
    t_stat_not_3n, p_value_not_3n = stats.ttest_ind(observed_ratios_plus_one_not_3n, expected_reciproc_ln_4_not_3n, equal_var=False)

    print("T-test for means (multiples of 3n): t-statistic =", t_stat_3n, ", p-value =", p_value_3n)
    print("T-test for means (not multiples of 3n): t-statistic =", t_stat_not_3n, ", p-value =", p_value_not_3n)"""

    
def main():
    min_interval = int(input("Enter the minimum interval: "))
    end_interval = int(input("Enter the maximum interval: "))
    start_time = time.time()
    data1=calculate(50000, 51000)
    end_time = time.time()
    start_time = time.time()
    data2=calculate(51001, 52000)
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time} seconds")
    datatxt_original = pd.read_csv('primes_data40000-50001.txt')
    data_series=[datatxt_original,data1,data2]
    datatxt_original=pd.concat(data_series)
    df=datatxt_original.sort_values(by=["Number"],ascending=True)
    df.to_csv(f'primes_data52000.txt',index=False)
if __name__ == "__main__":
    main()


[(50000, 50699), (50700, 51000)]
[(51001, 51700), (51701, 52000)]
Elapsed time: 384.2400724887848 seconds


In [3]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import argparse
from mpmath import li
import sympy
from functools import lru_cache
import numpy as np
#from scipy import stats



@lru_cache(maxsize=None)
def is_prime(num):
    return sympy.isprime(num)


def addition(number_to_generate_primes):
    """Generate primes based on the given number."""
    quotient = number_to_generate_primes
    secondmult = 0
    thirdmult = 0
    primes = []
    multiple_minus_one = 0
    twins_number = 0
    possible_twin1=0
    possible_twin2=0
    print("calculating", number_to_generate_primes)
    for index in range(2, number_to_generate_primes + 1):
        
        if((index & 1) == 0 or (quotient & 1) == 0):
            prime_plus_one = index * quotient + 1
            prime_minus_one = index * quotient - 1
            prime1=False
            prime2=False
            if (prime_plus_one&1==1 and is_prime(prime_plus_one)) or prime_plus_one==2:
                secondmult += 1
                prime1=True
                possible_twin1=prime_plus_one
                primes.append(prime_plus_one)

            if (prime_minus_one&1==1 and is_prime(prime_minus_one)) or prime_minus_one==2:
                multiple_minus_one += 1
                prime2=True
                possible_twin2=prime_minus_one
                primes.append(prime_minus_one)
            if(prime1==False or prime2==False):
                continue
            if((prime_plus_one-1) % 6!=0):
                continue
            if  (prime_plus_one-1) %6 ==0 and prime1 and prime2 and (possible_twin1 - possible_twin2 == 2):#all twin except 3,5 the composed number between the is multiple of 6n
                twins_number += 1# the 6n verification used for faster exit if not

    return secondmult, primes, multiple_minus_one, twins_number

def get_background_color(label):
    """Get the background color for a given label."""
    background_color_chart = [
        "rgba(0, 0, 0, 1.0)",  # black
        "rgba(0, 255, 0, 1.0)",  # Green (Cool)
        "rgba(255, 0, 0, 1.0)",  # Red (Warm)
        "rgba(0, 0, 255, 1.0)",  # Blue (Cool)
        "rgba(255, 0, 255, 1.0)",  # Magenta (Cool)
        "rgba(255, 255, 0, 1.0)",  # Yellow (Warm)
    ]
    background_color_chart_special = [
        "rgba(255, 128, 0, 1.0)",
        "rgba(64, 224, 208, 1.0)",
    ]

    remainder_420 = label % 210
    if remainder_420 == 0:
        return background_color_chart_special[1]
    remainder_60 = label % 30
    if remainder_60 == 0:
        return background_color_chart_special[0]
    remainder = label % 6

    if remainder == 1:
        return background_color_chart[0]  # black for 12n+2
    elif remainder == 2:
        return background_color_chart[1]  # Green for 12n+4
    elif remainder == 3:
        return background_color_chart[2]  # Red for 12n+6
    elif remainder == 4:
        return background_color_chart[3]  # Blue for 12n+8
    elif remainder == 5:
        return background_color_chart[4]  # Magenta for 12n+10
    elif remainder == 0:
        return background_color_chart[5]  # Yellow for 12n

    # Default color for any unexpected cases
    return "rgba(128, 128, 128, 1.0)"  # Gray

def calculate_range(start, end):
    """Calculate prime-related data for a range of numbers."""
    results = []
    for number in range(start, end + 1):
        result = addition(number)
        prime_check = is_prime(number)
        results.append((number, result, prime_check))
    return results

def calculate(min_interval, end_interval):
    """Perform the main calculations and generate the data for plotting."""
    unique_primes = []
    labels = []
    labels2 = []
    labels3 = []
    labels4 = []
    prime_count = []
    primes_count_plus = 0
    primes_count_minus = 0
    prime_count_minus_one_equation = []
    prime_twins_count = []
    prime_twins_exception = []
    prime_twins_exception3 = []
    prime_twins_exception4 = []
    prime_twins_ratio_plus_one = []
    prime_reverse_twins_ratio_plus_one = []
    ln_func = []
    ln_func2 = []
    ln_func3 = []
    ln_func4 = []
    ln = []
    labels_six_plus_one=[]
    values_six_plus_one=[]
    reciproc_ln_2=[]
    reciproc_ln_4=[]
    reciproc_ln_for_multiple_of_3 = []
    reciproc_ln = []
    is_number_prime=[]
    sqrt2 = math.sqrt(2)
    # Parallel processing
    chunk_size = 500  # Adjust the chunk size based on your system's capabilities
    ranges = [(i, min(i + chunk_size - 1, end_interval)) for i in range(min_interval, end_interval + 1, chunk_size)]
    print(ranges)


    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(calculate_range, start, end) for start, end in ranges]
        for future in as_completed(futures):
            results = future.result()
            for number, result, prime_check in results:
                if number > 0:  # Ensure number is positive
                    labels.append(number)
                    prime_count.append(result[0])
                    primes_count_plus += result[0]
                    primes_count_minus += result[2]
                    prime_count_minus_one_equation.append(result[2])
                    prime_twins_count.append(result[3])
                    is_number_prime.append(prime_check)
                    #print("Calculating ", number)
                    """#print(number,result[0],result[4])
                    math_log=math.log(number)
                    divide_log=round(number / math_log)
                    ln_func.append(divide_log)
                    ln_func2.append(divide_log * sqrt2)
                    ln_func3.append(divide_log / sqrt2)
                    ln_func4.append(divide_log * 2 * sqrt2)
                    log_number = math_log - np.pi
                    log_number_pi_2 = math_log - 2*math.sqrt(2)
                    log_number_pi_4 = math_log + 2*math.sqrt(2)
                    result_log_twin_prime_ratio = log_number
                    
                    ln.append(result_log_twin_prime_ratio)
                    reciproc_ln_for_multiple_of_3.append(result_log_twin_prime_ratio - 2)
                    reciproc_ln.append(1 / result_log_twin_prime_ratio)
                    reciproc_ln_2.append(1 / log_number_pi_2)
                    reciproc_ln_4.append(1 / log_number_pi_4)
                    if(result[3]>0):
                        prime_twins_ratio_plus_one.append(result[0] / result[3])
                    else:
                        prime_twins_ratio_plus_one.append(0)
                    prime_reverse_twins_ratio_plus_one.append(result[3] / result[0])

                    if number % 6 == 0 and result[3] > 0:
                        labels2.append(number)
                        prime_twins_exception.append(result[3])
                    if number % 30 == 0 and result[3] > 0:
                        labels3.append(number)
                        prime_twins_exception3.append(result[3])
                    if number % 210 == 0 and result[3] > 0:
                        labels4.append(number)
                        prime_twins_exception4.append(result[3])
                    if number % 6 == 1 and result[3] > 0:
                        labels_six_plus_one.append(number)
                        values_six_plus_one.append(result[3])"""
                    

                    

    data_to_csv={
        'Number':labels,
        'Primes +1':prime_count,
        'Primes -1':prime_count_minus_one_equation,
        'Twins':prime_twins_count,
        'isPrime':is_number_prime,
    }
    df_new=pd.DataFrame(data_to_csv).sort_values(by=["Number"],ascending=True)
    df_new.to_csv(f'primes_data{min_interval}-{end_interval}.csv',index=False)
    return df_new
    

def main():
    #min_interval = int(input("Enter the minimum interval: "))
    #end_interval = int(input("Enter the maximum interval: "))
    
    a=60001
    b=a+999
    c=b+1
    d=c+999
    to_read=a-1
    datatxt_original = pd.read_csv(f'primes_data{to_read}.txt')
    start_time = time.time()
    data1=calculate(a, b)
    end_time = time.time()
    start_time = time.time()
    data2=calculate(c, d)
    end_time = time.time()
    print(f"Elapsed time: {end_time - start_time} seconds")

    data_series=[datatxt_original,data1,data2]
    datatxt_original=pd.concat(data_series)
    df=datatxt_original.sort_values(by=["Number"],ascending=True).drop_duplicates()
    df.to_csv(f'primes_data{d}.txt',index=False)
if __name__ == "__main__":
    main()


[(60001, 60700), (60701, 61000)]
calculatingcalculating  6000160701

calculating 60702
calculating 60002
calculating 60003
calculating 60703
calculating 60004
calculating 60704
calculating 60005
calculating 60705
calculating 60006
calculating 60706
calculating 60007
calculating 60707
calculating 60008
calculating 60708
calculating 60009
calculating 60709
calculating 60010
calculating 60710
calculating 60011
calculating 60711
calculating 60012
calculating 60712
calculating 60013
calculating 60713
calculating 60014
calculating 60714
calculating 60015
calculating 60715
calculating 60016
calculating 60716
calculating 60017
calculating 60717
calculating 60018
calculating 60718
calculating 60019
calculating 60719
calculating 60020
calculating 60720
calculating 60021
calculating 60721
calculating 60022
calculating 60722
calculating 60023
calculating 60024
calculating 60723
calculating 60724
calculating 60025
calculating 60725
calculating 60026
calculating 60726
calculating 60027
calculating 6

In [7]:
df=pd.read_csv("primes_data55002.txt").sort_values(by="Number",ascending=True)
df=df.drop_duplicates()
df.to_csv(f'primes_data55004.txt',index=False)

In [2]:
import pandas as pd
datatxt_original = pd.read_csv('primes_data56000.txt')
data1=pd.read_csv("primes_data56001-57000.csv")
data2=pd.read_csv("primes_data57001-58000.csv")
data3=pd.read_csv("primes_data58001-59000.csv")
data4=pd.read_csv("primes_data59001-60000.csv")
data_series=[datatxt_original,data1,data2,data3,data4]
datatxt_original=pd.concat(data_series)
df=datatxt_original.sort_values(by=["Number"],ascending=True).drop_duplicates()
df.to_csv(f'primes_data60000.txt',index=False)

In [2]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
import argparse
from mpmath import li
import sympy
from functools import lru_cache
import numpy as np
from scipy import stats
datatxt_original = pd.read_csv('primes_data40000-50001.txt')
df=datatxt_original.sort_values(by=["Number"],ascending=True)
df.to_csv('primes_data40000-50000.txt',index=False)

In [2]:
import pandas as pd
import plotly.graph_objects as go

# Read the CSV file
data = pd.read_csv('primes_data58000.txt')

# Count occurrences of each number in the 'Number' column
number_counts_primes_plus_one = data['Primes +1'].value_counts().sort_index()
number_counts_primes_minus_one = data['Primes -1'].value_counts().sort_index()
number_counts_primes_twins = data['Twins'].value_counts().sort_index()
data2 = pd.read_csv('primes_data40000-40500.csv')

# Count occurrences of each number in the specified columns for the second dataset
number_counts_primes_plus_one_data2 = data2['Primes +1'].value_counts().sort_index()
number_counts_primes_minus_one_data2 = data2['Primes -1'].value_counts().sort_index()
number_counts_primes_twins_data2 = data2['Twins'].value_counts().sort_index()

# Add the counts from the second dataset to the counts from the first dataset
number_counts_primes_plus_one = number_counts_primes_plus_one.add(number_counts_primes_plus_one_data2, fill_value=0)
number_counts_primes_minus_one = number_counts_primes_minus_one.add(number_counts_primes_minus_one_data2, fill_value=0)
number_counts_primes_twins = number_counts_primes_twins.add(number_counts_primes_twins_data2, fill_value=0)


print("Primes +1 Counts:\n", number_counts_primes_plus_one)
print("Primes -1 Counts:\n", number_counts_primes_minus_one)
print("Twins Counts:\n", number_counts_primes_twins)
# Plot the results using Plotly
fig = go.Figure(data=[go.Scatter(x=number_counts_primes_plus_one.index, y=number_counts_primes_plus_one.values,mode="lines",
     line=dict(color="red"))])

# Customize the plot
fig.update_layout(
    title='Occurrences of Each Number of primes +1 generated',
    xaxis_title='Number',
    yaxis_title='Count'
)

# Show the plot
fig.show()

fig_primes_minus_one = go.Figure(data=[go.Scatter(x=number_counts_primes_minus_one.index, y=number_counts_primes_minus_one.values,mode="lines",
     line=dict(color="red"))])

# Customize the plot
fig_primes_minus_one.update_layout(
    title='Occurrences of Each  Number primes -1 generated',
    xaxis_title='Number',
    yaxis_title='Count'
)

# Show the plot
fig_primes_minus_one.show()


fig_twins = go.Figure(data=[go.Scatter(
    x=number_counts_primes_twins.index,
     y=number_counts_primes_twins.values,
     mode="lines",
     line=dict(color="red")
     )])

# Customize the plot
fig_twins.update_layout(
    title='Occurrences of Each Number of Twins generated',
    xaxis_title='Number',
    yaxis_title='Count'
)

# Show the plot
fig_twins.show()

FileNotFoundError: [Errno 2] No such file or directory: 'primes_data40000-40500.csv'

In [4]:
import pandas as pd
import plotly.graph_objects as go

# Read the CSV file
datatxt_all = pd.read_csv('primes_data62000.txt')
datatxt=datatxt_all

# Count occurrences of each number in the 'Number' column
number_counts_primes_plus_one = datatxt['Primes +1'].value_counts().sort_index()
number_counts_primes_minus_one = datatxt['Primes -1'].value_counts().sort_index()
number_counts_primes_twins = datatxt['Twins'].value_counts().sort_index()
"""data2 = pd.read_csv('primes_data40000-40500 copy.txt')

# Count occurrences of each number in the specified columns for the second dataset
number_counts_primes_plus_one_data2 = data2['Primes +1'].value_counts().sort_index()
number_counts_primes_minus_one_data2 = data2['Primes -1'].value_counts().sort_index()
number_counts_primes_twins_data2 = data2['Twins'].value_counts().sort_index()

# Add the counts from the second dataset to the counts from the first dataset
number_counts_primes_plus_one = number_counts_primes_plus_one.add(number_counts_primes_plus_one_data2, fill_value=0)
number_counts_primes_minus_one = number_counts_primes_minus_one.add(number_counts_primes_minus_one_data2, fill_value=0)
number_counts_primes_twins = number_counts_primes_twins.add(number_counts_primes_twins_data2, fill_value=0)"""



# Plot the results using Plotly
fig = go.Figure(data=[go.Scatter(x=number_counts_primes_plus_one.index, y=number_counts_primes_plus_one.values,mode="markers",
     line=dict(color="red"))])

# Customize the plot
fig.update_layout(
    title='Occurrences of Each Number of primes +1 generated',
    xaxis_title='Number',
    yaxis_title='Count'
)

# Show the plot
fig.show()

fig_primes_minus_one = go.Figure(data=[go.Scatter(x=number_counts_primes_minus_one.index, y=number_counts_primes_minus_one.values,mode="lines",
     line=dict(color="red"))])

# Customize the plot
fig_primes_minus_one.update_layout(
    title='Occurrences of Each  Number primes -1 generated',
    xaxis_title='Number',
    yaxis_title='Count'
)

# Show the plot
fig_primes_minus_one.show()


fig_twins = go.Figure(data=[go.Scatter(
    x=number_counts_primes_twins.index,
     y=number_counts_primes_twins.values,
     mode="markers",
     line=dict(color="red")
     )])

# Customize the plot
fig_twins.update_layout(
    title='Occurrences of Each Number of Twins generated',
    xaxis_title='Number',
    yaxis_title='Count'
)

# Show the plot
fig_twins.show()

is this it ? first the sqkwed normal distrubition persist and what could be another pattern is when the points discontinue in the twins it manifist the inverse logarthim in the primes

In [100]:
import sympy
import math

def is_prime(n):
    return sympy.isprime(n)

def find_twin_primes_your_method(number,limit):
    twin_primes = []
    primes_found=[]
    maximum=0
    
    for n in range(number, limit, number):
        if n & 1 == 0 and not is_prime(n):
            a = n // 2
            p1 = a * n - 1
            p2 = a * n + 1
            is_prime1=is_prime(p1)
            is_prime2=is_prime(p2)
            if(is_prime1):
                primes_found.append(p1)
            if(is_prime2):
                primes_found.append(p2)
            if(is_prime1 and p1>maximum):
                maximum=p1
            if(is_prime2 and p2>maximum):
                maximum=p2
            if is_prime1 and is_prime2 and p2-p1==2:
                twin_primes.append((p1, p2))
    return twin_primes,primes_found,maximum
number=13082761331670030*47*53*59*61*67*71*73*79*83*89*93*97*101*103*107*109*113*127*131*137*139*149*151*157
limit = 10**68
print("in the range between the promodial : ",number, " and ",limit)

twin_primes_your_method,primes_found,maximum = find_twin_primes_your_method(number,limit)
print("maximum prime found: ",maximum)
print("number of print found:",len(primes_found)," min prime: ",primes_found[0], "max prime: ",primes_found[-1])
if(len(twin_primes_your_method)>0):
    print("Your Method Twin Primes between:", twin_primes_your_method[0]," and ",twin_primes_your_method[-1])

import time

def test_method(method, limit):
    start_time = time.time()
    twin_primes,prime_found,maximum = method(number,limit)
    end_time = time.time()
    return len(twin_primes), end_time - start_time,maximum,twin_primes


# Test your method
count_your_method, time_your_method,maximum,twin_primes = test_method(find_twin_primes_your_method, limit)

print("Your Method - Count:", count_your_method, "Time:", time_your_method)
print("primodial number and the strat=",number," the end:",limit )
data_str = [(str(a), str(b)) for a, b in twin_primes]

data_primes_to_csv={
        'prime found':[(str(a)) for a in primes_found],
    }
import pandas as pd
df=pd.DataFrame(data_primes_to_csv)
max_digit_prime=len(str(primes_found[-1]))
max_digit_twin=len(str(twin_primes[-1][-1]))
min_digit_prime=len(str(primes_found[0]))
min_digit_twin=len(str(twin_primes[0][0]))
length_primes=len(primes_found)
length_twin=len(twin_primes)
df.to_csv(f'primesCount_{length_primes}_Between_{min_digit_prime}_and_{max_digit_prime}_digits.txt',index=False,sep=",")
df = pd.DataFrame(data_str, columns=['Twin 1', 'Twin 2'])

# Write the DataFrame to a CSV file
df.to_csv(f'TwinsCount_{length_twin}_Between_{min_digit_twin}_and_{max_digit_twin}_digits.txt.txt', index=False,sep=",")

print("Data has been written to output.csv")



in the range between the promodial :  3289890530415727020179126339179512211721354070253767900285977910  and  100000000000000000000000000000000000000000000000000000000000000000000
maximum prime found:  4998306454415559720398348983787316213332182646302273390314609498428974301993457700656121743125750085348740766951854230859851002398548049
number of print found: 1834  min prime:  2386555224317255745045928935945845097064527131707794539873619182241761813604584024840326678725105164096119484745219480984256966049 max prime:  4998306454415559720398348983787316213332182646302273390314609498428974301993457700656121743125750085348740766951854230859851002398548049
Your Method Twin Primes between: (37831695062636542308615412847978034853679102364186895171197154348272210915625522239174979511845865701661337737664074044485171441760799, 37831695062636542308615412847978034853679102364186895171197154348272210915625522239174979511845865701661337737664074044485171441760801)  and  (4673830530586252203671251732

In [98]:

data_str = [(str(a), str(b)) for a, b in twin_primes]

data_primes_to_csv={
        'prime found':[(str(a)) for a in primes_found],
    }
import pandas as pd
df=pd.DataFrame(data_primes_to_csv)
max_digit_prime=len(str(primes_found[-1]))
max_digit_twin=len(str(twin_primes[-1][-1]))
min_digit_prime=len(str(primes_found[0]))
min_digit_twin=len(str(twin_primes[0][0]))
length_primes=len(primes_found)
length_twin=len(twin_primes)
df.to_csv(f'primesCount_{length_primes}_Between_{min_digit_prime}_and_{max_digit_prime}_digits.txt',index=False,sep=",")
df = pd.DataFrame(data_str, columns=['Twin 1', 'Twin 2'])


# Write the DataFrame to a CSV file
df.to_csv(f'TwinsCount_{length_twin}_Between_{min_digit_twin}_and_{max_digit_twin}_digits.txt.txt', index=False,sep=",")

print("Data has been written to output.csv")

Data has been written to output.csv
